In [66]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import numpy as np
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

from pathlib import Path

p = Path(r"D:\Projektit\Koodausprojekteja\Math test\archive")
df = pd.DataFrame()
files = p.rglob('*.csv')

df = pd.concat([pd.read_csv(file) for file in files]).dropna()

# print(df)

df2 = df[['Order ID','Product','Quantity Ordered']]
df2['Quantity Ordered'] = pd.to_numeric(df2['Quantity Ordered'], errors='coerce')
print(df2)


# df2 = df[['userId','movieId','rating']]
# print(df_name.drop_duplicates(keep='last'))
# 

print(df2.dtypes)

      Order ID                     Product  Quantity Ordered
0       176558        USB-C Charging Cable               2.0
2       176559  Bose SoundSport Headphones               1.0
3       176560                Google Phone               1.0
4       176560            Wired Headphones               1.0
5       176561            Wired Headphones               1.0
...        ...                         ...               ...
11681   259353      AAA Batteries (4-pack)               3.0
11682   259354                      iPhone               1.0
11683   259355                      iPhone               1.0
11684   259356      34in Ultrawide Monitor               1.0
11685   259357        USB-C Charging Cable               1.0

[186305 rows x 3 columns]
Order ID             object
Product              object
Quantity Ordered    float64
dtype: object


C:\Users\Maxca\AppData\Local\Temp\ipykernel_14228\3262830765.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Quantity Ordered'] = pd.to_numeric(df2['Quantity Ordered'], errors='coerce')


In [64]:
pivotDf = pd.pivot_table(df2,index = 'Order ID',columns = 'Product',values = 'Quantity Ordered',aggfunc = 'sum').reset_index().fillna(0).drop('Order ID', axis=1)
pivotDf = pivotDf.T.dot(pivotDf)

cosScore = pd.DataFrame(cosine_similarity(pivotDf))
cosScore.index = pivotDf.index
cosScore.columns = np.array(pivotDf.index)
df_pairs = cosScore.stack().reset_index()
# print(df_pairs)
df_pairs = df_pairs.rename(columns={'Product': 'ProductID1', 'level_1': 'ProductID2', 0: 'Score'})
df_pairs.reset_index(drop=True, inplace=True, level=0)
df_pairs = df_pairs[df_pairs['Score'] > 0.2].sort_values('Score', ascending=False)
print(df_pairs.to_excel('test4.xlsx'))

None


In [65]:
print(df_pairs[df_pairs['ProductID1'] == 'Google Phone'].sort_values('Score', ascending=False))

       ProductID1            ProductID2     Score
189  Google Phone          Google Phone  1.000000
196  Google Phone  USB-C Charging Cable  0.228121


In [28]:


df_p = df_pairs.copy()

df_p = df_p.merge(df_names, how='inner', left_on='ProductID1', right_on='Product').rename(columns={'Product Name':'ProductName1'}).drop(columns='Product ID')
df_p = df_p.merge(df_names, how='inner', left_on='ProductID2', right_on='Product ID').rename(columns={'Product Name':'ProductName2'}).drop(columns='Product ID')



print(df_p)

             ProductID1       ProductID2  Score                                       ProductName1                                       ProductName2
0       TEC-PH-10004977  TEC-PH-10004977    1.0                                        GE 30524EE4                                        GE 30524EE4
1       TEC-PH-10004977  TEC-PH-10004977    1.0                                        GE 30524EE4                                        GE 30524EE4
2       TEC-PH-10004977  TEC-PH-10004977    1.0                                        GE 30524EE4                                        GE 30524EE4
3       TEC-PH-10004977  TEC-PH-10004977    1.0                                        GE 30524EE4                                        GE 30524EE4
4       TEC-PH-10004977  TEC-PH-10004977    1.0                                        GE 30524EE4                                        GE 30524EE4
...                 ...              ...    ...                                                ...  

In [14]:
print(df_names)

      movieId                                      title                                       genres
0           1                           Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1           2                             Jumanji (1995)                   Adventure|Children|Fantasy
2           3                    Grumpier Old Men (1995)                               Comedy|Romance
3           4                   Waiting to Exhale (1995)                         Comedy|Drama|Romance
4           5         Father of the Bride Part II (1995)                                       Comedy
...       ...                                        ...                                          ...
9737   193581  Black Butler: Book of the Atlantic (2017)              Action|Animation|Comedy|Fantasy
9738   193583               No Game No Life: Zero (2017)                     Animation|Comedy|Fantasy
9739   193585                               Flint (2017)                          

In [29]:


print(df_p[df_p['ProductID1'] == 'OFF-AR-10004042'].sort_values('Score', ascending=False).to_excel('test3.xlsx'))

None
